# 02 - UDA-Hub Core Database Setup

This notebook sets up the UDA-Hub core database including:
1. SQLite database for accounts, users, tickets, and knowledge articles
2. Chroma vector store for RAG semantic search

**Run this AFTER `01_external_db_setup.py`**

## Cell 1: Setup Working Directory

**CRITICAL**: This ensures all relative paths work correctly regardless of where you opened the notebook.

In [ ]:
import os

# Get the directory where this notebook is located
# This handles both Jupyter and VS Code scenarios
try:
    # For Jupyter notebooks
    notebook_dir = os.path.dirname(os.path.abspath('__file__'))
except:
    notebook_dir = os.getcwd()

# If we're not in the starter directory, try to find it
if not os.path.exists('data/external/cultpass_articles.jsonl'):
    # Look for starter directory
    possible_paths = [
        os.path.join(notebook_dir, 'starter'),
        os.path.dirname(notebook_dir),
        notebook_dir
    ]
    
    for path in possible_paths:
        test_file = os.path.join(path, 'data/external/cultpass_articles.jsonl')
        if os.path.exists(test_file):
            os.chdir(path)
            print(f"✅ Changed to: {path}")
            break
    else:
        print("⚠️ Could not find project directory automatically.")
        print("Please set your working directory manually:")
        print("os.chdir('/path/to/your/starter')")

print(f"\n📁 Current working directory: {os.getcwd()}")
print(f"📄 Data files exist: {os.path.exists('data/external/cultpass_articles.jsonl')}")

## Cell 2: Ensure Required Directories Exist

In [ ]:
import os

# Create required directories
directories = [
    'data/core',
    'data/core/chroma_db'
]

for dir_path in directories:
    os.makedirs(dir_path, exist_ok=True)
    print(f"✅ Directory ready: {dir_path}")

print(f"\n📁 Absolute path for chroma_db: {os.path.abspath('data/core/chroma_db')}")

## Cell 3: Initialize SQLite Database

In [ ]:
import json
import uuid
from sqlalchemy import create_engine

from utils import reset_db, get_session, model_to_dict
from data.models import udahub

print("=" * 60)
print("Setting up UDA-Hub Core Database")
print("=" * 60)

# Initialize database
udahub_db = "data/core/udahub.db"
reset_db(udahub_db)

engine = create_engine(f"sqlite:///{udahub_db}", echo=False)
udahub.Base.metadata.create_all(engine)

print(f"\n✅ Database created at: {os.path.abspath(udahub_db)}")

## Cell 4: Create Account

In [ ]:
print("\n🏢 Creating Account...")

account_id = "cultpass"
account_name = "CultPass Card"

with get_session(engine) as session:
    account = udahub.Account(
        account_id=account_id,
        account_name=account_name,
    )
    session.add(account)

print(f"✅ Created account: {account_name} ({account_id})")

## Cell 5: Load Knowledge Base Articles

In [ ]:
print("\n📚 Loading Knowledge Base Articles...")

cultpass_articles = []
with open('data/external/cultpass_articles.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        cultpass_articles.append(json.loads(line))

print(f"✅ Loaded {len(cultpass_articles)} articles")

# Verify we have at least 14 articles
if len(cultpass_articles) < 14:
    raise AssertionError("You should load the articles with at least 14 records")

# Store in SQLite
with get_session(engine) as session:
    kb = []
    for article in cultpass_articles:
        knowledge = udahub.Knowledge(
            article_id=str(uuid.uuid4()),
            account_id=account_id,
            title=article["title"],
            content=article["content"],
            tags=article["tags"]
        )
        kb.append(knowledge)
    session.add_all(kb)

print(f"✅ Stored {len(kb)} articles in database")

## Cell 6: Initialize RAG Vector Store (Chroma + OpenAI)

This is where the Chroma database gets created with OpenAI embeddings.

In [ ]:
print("\n🔍 Initializing RAG Vector Store...")
print("   - Using OpenAI embeddings (text-embedding-ada-002)")
print("   - Storing in Chroma vector database")

# Import AFTER setting up the working directory
from agentic.tools.rag_tools import initialize_vector_store, CHROMA_PERSIST_DIR

print(f"\n📁 Chroma will be stored at: {CHROMA_PERSIST_DIR}")
print(f"📁 Parent directory exists: {os.path.exists(os.path.dirname(CHROMA_PERSIST_DIR))}")

# Initialize vector store with the articles
# This creates embeddings and stores them in Chroma
initialize_vector_store(cultpass_articles)

print("\n✅ Semantic search is now enabled!")

## Cell 7: Create Sample Ticket

In [ ]:
print("\n🎫 Creating Sample Ticket...")

# Load cultpass users for reference
cultpass_users = []
with open('data/external/cultpass_users.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        cultpass_users.append(json.loads(line))

# Sample ticket data
ticket_info = {
    "status": "open",
    "content": "I can't log in to my Cultpass account.",
    "owner_id": cultpass_users[0]["id"],
    "owner_name": cultpass_users[0]["name"],
    "role": "user",
    "channel": "chat",
    "tags": "login, access",
}

with get_session(engine) as session:
    # Create or get user
    user = session.query(udahub.User).filter_by(
        account_id=account_id,
        external_user_id=ticket_info["owner_id"],
    ).first()
    
    if not user:
        user = udahub.User(
            user_id=str(uuid.uuid4()),
            account_id=account_id,
            external_user_id=ticket_info["owner_id"],
            user_name=ticket_info["owner_name"],
        )
    
    # Create ticket
    ticket = udahub.Ticket(
        ticket_id=str(uuid.uuid4()),
        account_id=account_id,
        user_id=user.user_id,
        channel=ticket_info["channel"],
    )
    
    # Create metadata
    metadata = udahub.TicketMetadata(
        ticket_id=ticket.ticket_id,
        status=ticket_info["status"],
        main_issue_type=None,
        tags=ticket_info["tags"],
    )
    
    # Create first message
    first_message = udahub.TicketMessage(
        message_id=str(uuid.uuid4()),
        ticket_id=ticket.ticket_id,
        role=udahub.RoleEnum.user,
        content=ticket_info["content"],
    )
    
    session.add_all([user, ticket, metadata, first_message])

print(f"✅ Created sample ticket for user: {ticket_info['owner_name']}")

## Cell 8: Verification Tests

In [ ]:
print("\n" + "=" * 60)
print("Verification Tests")
print("=" * 60)

print("\n🏢 Account:")
with get_session(engine) as session:
    account = session.query(udahub.Account).filter_by(
        account_id=account_id
    ).first()
    print(account)

print("\n📚 Knowledge Articles:")
with get_session(engine) as session:
    account = session.query(udahub.Account).filter_by(
        account_id=account_id
    ).first()
    for article in account.knowledge_articles:
        print(article)

print("\n👤 Users:")
with get_session(engine) as session:
    users = session.query(udahub.User).all()
    for user in users:
        print(user)

print("\n🎫 Tickets:")
with get_session(engine) as session:
    user = session.query(udahub.User).filter_by(
        account_id=account_id,
        external_user_id=ticket_info["owner_id"],
    ).first()
    
    ticket = user.tickets[0]
    print(f"Ticket: {ticket}")
    print(f"Metadata: {ticket.ticket_metadata}")
    for message in ticket.messages:
        print(f"Message: {message}")

print("\n✅ UDA-Hub database setup complete!")

## Cell 9: Test RAG Search

In [ ]:
print("\n🔍 Testing RAG Semantic Search...")

from agentic.tools.rag_tools import search_knowledge_base

# Test query
test_query = "How do I reset my password?"
print(f"\nQuery: {test_query}")
print("-" * 40)

result = search_knowledge_base.invoke(test_query)
print(result[:500] + "..." if len(result) > 500 else result)

## Cell 10: Verify Files Created

In [ ]:
print("\n📁 Files created:")
print("-" * 40)

files_to_check = [
    'data/core/udahub.db',
    'data/core/chroma_db'
]

for f in files_to_check:
    exists = os.path.exists(f)
    status = "✅" if exists else "❌"
    print(f"{status} {f}: {exists}")
    
    if os.path.isdir(f):
        contents = os.listdir(f)
        print(f"   Contents: {contents}")

print("\n🎉 Setup complete! You can now run 03_agentic_app.ipynb")